This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Modern convnet architecture patterns

### Modularity, hierarchy, and reuse

### Residual connections

**Residual block where the number of filters changes**

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)
# 残差
residual = x
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
residual = layers.Conv2D(64, 1)(residual)
x = layers.add([x, residual])

**Case where target block includes a max pooling layer**

In [2]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)
residual = x
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(2, padding="same")(x)
# 在残差投影中使用strides=2，以匹配最大汇聚层导致的下采样
residual = layers.Conv2D(64, 1, strides=2)(residual)
x = layers.add([x, residual])

In [3]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Rescaling(1./255)(inputs)

def residual_block(x, filters, pooling=False):
    residual = x
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    if pooling:
        # 如果使用最大汇聚，则添加一个步进卷积，将残差投影为想要的形状
        x = layers.MaxPooling2D(2, padding="same")(x)
        residual = layers.Conv2D(filters, 1, strides=2)(residual)
    elif filters != residual.shape[-1]:
        residual = layers.Conv2D(filters, 1)(residual)
    x = layers.add([x, residual])
    return x

x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=False)

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_3[0][0]']                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 32, 32, 32)   896         ['rescaling[0][0]']              
                                                                                                  
 conv2d_7 (Conv2D)              (None, 32, 32, 32)   9248        ['conv2d_6[0][0]']               
                                                                                              

### Batch normalization

### Depthwise separable convolutions

### Putting it together: A mini Xception-like model

In [4]:
import os, shutil, pathlib
from tensorflow.keras.utils import image_dataset_from_directory

new_base_dir = pathlib.Path("../data/cats_vs_dogs_small")

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [5]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [6]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)

x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32, 64, 128, 256, 512]:
    residual = x

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    # 由于后面会将Conv2D层的输出规范化，所以该层不需要偏置向量
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    # 将激活放在BN之后
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
    
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(
        size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])

x = layers.GlobalAveragePooling2D()(x)
# 添加dropout层进行正则化
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [7]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset)

Epoch 1/100
63/63 [==============================] - 25s 303ms/step - loss: 0.7080 - accuracy: 0.5685 - val_loss: 0.6946 - val_accuracy: 0.5000
Epoch 2/100
63/63 [==============================] - 19s 298ms/step - loss: 0.6596 - accuracy: 0.6080 - val_loss: 0.6987 - val_accuracy: 0.5000
Epoch 3/100
63/63 [==============================] - 19s 300ms/step - loss: 0.6455 - accuracy: 0.6230 - val_loss: 0.7148 - val_accuracy: 0.5000
Epoch 4/100
63/63 [==============================] - 19s 300ms/step - loss: 0.6232 - accuracy: 0.6395 - val_loss: 0.7081 - val_accuracy: 0.5000
Epoch 5/100
63/63 [==============================] - 19s 303ms/step - loss: 0.6069 - accuracy: 0.6680 - val_loss: 0.7315 - val_accuracy: 0.5000
Epoch 6/100
63/63 [==============================] - 19s 302ms/step - loss: 0.5774 - accuracy: 0.6975 - val_loss: 0.7438 - val_accuracy: 0.5000
Epoch 7/100
63/63 [==============================] - 19s 305ms/step - loss: 0.5718 - accuracy: 0.7125 - val_loss: 0.8261 - val_accuracy:

Epoch 41/100
63/63 [==============================] - 19s 307ms/step - loss: 0.2222 - accuracy: 0.9150 - val_loss: 0.8976 - val_accuracy: 0.7400
Epoch 42/100
63/63 [==============================] - 19s 309ms/step - loss: 0.2080 - accuracy: 0.9130 - val_loss: 0.9678 - val_accuracy: 0.6470
Epoch 43/100
63/63 [==============================] - 19s 307ms/step - loss: 0.1999 - accuracy: 0.9115 - val_loss: 0.7147 - val_accuracy: 0.7870
Epoch 44/100
63/63 [==============================] - 20s 311ms/step - loss: 0.2024 - accuracy: 0.9200 - val_loss: 0.5173 - val_accuracy: 0.8130
Epoch 45/100
63/63 [==============================] - 19s 307ms/step - loss: 0.2048 - accuracy: 0.9130 - val_loss: 0.7294 - val_accuracy: 0.7860
Epoch 46/100
63/63 [==============================] - 28s 446ms/step - loss: 0.2121 - accuracy: 0.9050 - val_loss: 0.3828 - val_accuracy: 0.8510
Epoch 47/100
63/63 [==============================] - 20s 312ms/step - loss: 0.1867 - accuracy: 0.9200 - val_loss: 0.5419 - val_ac

63/63 [==============================] - 20s 312ms/step - loss: 0.0990 - accuracy: 0.9680 - val_loss: 1.9785 - val_accuracy: 0.7180
Epoch 98/100
63/63 [==============================] - 20s 311ms/step - loss: 0.0929 - accuracy: 0.9660 - val_loss: 0.4897 - val_accuracy: 0.8690
Epoch 99/100
63/63 [==============================] - 20s 309ms/step - loss: 0.0805 - accuracy: 0.9715 - val_loss: 0.4183 - val_accuracy: 0.8720
Epoch 100/100
63/63 [==============================] - 19s 308ms/step - loss: 0.0858 - accuracy: 0.9665 - val_loss: 0.5403 - val_accuracy: 0.8740
